# Runnning Workout Scheduler

## Imports

In [49]:
from ortools.sat.python import cp_model
import math

## Create Variables

In [50]:
cycle = 3
day = 10
session = ['e', 'THRESH', 'LR', 'rest', f'e \t hill', f'e \t strides']
all_days = range(day)
all_cycles = range(cycle)
all_sessions = range(len(session))
workoutList = [1,2]

In [51]:
model = cp_model.CpModel()

In [52]:
calendar = {}
for c in all_cycles:
    for d in all_days:
        for s in all_sessions:
            calendar[(c, d, s)] = model.NewBoolVar(f'cycle{c}day{d}session{s}')

# print(len(calendar))

## Utility Methods

In [53]:
def next_day(calendar_tuple, change):
    (c, d, s) = calendar_tuple
    if d + change < day:
        return (c, d + 1, s)
    else:
        (change, d) = divmod(d + change, day)
        if (c + change < cycle):
            return (c + change, d, s)
    return calendar_tuple # if last day of last cycle return itself


In [54]:
def prev_day(calendar_tuple, change):
    # change: how many days you want to go back
    (c, d, s) = calendar_tuple
    if d - change >= 0:
        return (c, d - change, s)
    else:
        change = d - change # definately negative
        while (change < 0 and c - 1 >= 0):
            change = day + change # decreases
            c = c - 1 # go back a cycle
        if c >= 0:
            return (c, change, s)
    return (0, 0, 0)

In [55]:
def day_counter(calendar_tuple):
    (c, d, s) = calendar_tuple
    return (10 * c) + d

In [56]:
def distance_btw_sessions(tuple1, tuple2):
    (c1, d1, s1) = tuple1
    (c2, d2, s2) = tuple2
    if s1 != s2:
        return None
    else:
        return abs(day_counter(tuple1) - day_counter(tuple2))

In [57]:
def limit_num_sessions_per_cycle(session_value, lb, ub):
    #Ex) I want 2 to 3 sessions per cycle in threshold
    for c in all_cycles:
        model.Add(sum(calendar[(c, d, session_value)] for d in all_days) <= ub)
        model.Add(sum(calendar[(c, d, session_value)] for d in all_days) >= lb)

In [58]:
def make_value_list(day):
    length = day / 2
    left = math.floor(length) 
    right = math.ceil(length)
    shift = 1 if (left < right) else 0
    left_list = list(reversed(range(left + shift)))
    right_list = list(range(right))
    return left_list + right_list

# print(make_value_list(10))

## Constraints

In [59]:
# Each day must have a session
for c in all_cycles:
    for d in all_days:
        model.Add(sum(calendar[(c, d, s)] for s in all_sessions) == 1)

In [60]:
# You can't have two workouts on consecutive days
for c in all_cycles:
    for d in all_days:
        model.Add(sum(calendar[(c, d, s)] + calendar[next_day((c, d, s), 1)] for s in workoutList) <= 1)
        # Don't include 'rest' in workout list because allowed to have rest day after workout

In [61]:
# You can't have workout after a rest day or a rest day after a rest day
# in this way a rest day behaves the same way as a workout
# except you are allowed to have a rest day after a workout
rList = [3] + workoutList
for c in all_cycles:
    for d in all_days:
        model.AddAtMostOne([calendar[(c, d, 3)]] + [calendar[next_day((c, d, r), 1)] for r in rList])


In [62]:
limit_num_sessions_per_cycle(1, 3, 4) #THRESH
limit_num_sessions_per_cycle(2, 1, 1) #LR
limit_num_sessions_per_cycle(3, 1, 1) #REST #Can't restict to one if set rest to happen every 5 days
limit_num_sessions_per_cycle(4, 1, 2) #Easy and hills
limit_num_sessions_per_cycle(5, 1, 3) #Easy and Strides

In [63]:
# Maximize the space between e + hill days
# value = make_value_list(day)
# for c in all_cycles:
#     model.Maximize(sum(calendar[(c, d, 4)] * value[d] for d in all_days))
#     model.Maximize(sum(calendar[(c, d, 5)] * value[d] for d in all_days))

In [64]:
# maximize num of workouts
# for c in all_cycles:
#     for d in all_days:
#         model.Maximize(sum(calendar[(c, d, w)] for w in workoutList))

In [65]:
# minimize num of easy runs
# for c in all_cycles:
#     model.Minimize(sum(calendar[(c, d, 0)] for d in all_days))

### Decision Strategy

In [66]:
#Add decision strategy: Choose all the Rest Days first
imp_days = []
for c in all_cycles:
    for d in all_days:
        for s in [3]:
            imp_days.append(calendar.get((c, d, s)))

model.AddDecisionStrategy(imp_days, cp_model.CHOOSE_FIRST, cp_model.SELECT_MAX_VALUE)

### Objective

In [67]:
#Maximize a cycle's value based on the types of sessions and what day they fall on and the parity of the cycle
score = []
score2 = []
for c in all_cycles:
    for d in all_days:
        score.append((c, d, 2)) #LR
        score2.append((c, d, 3)) #Rest

model.Maximize(sum((day_counter(s) % day) * calendar[s] for s in score)) # Maximize LR to be later in a cycle
model.Maximize(sum((day - (day_counter(s) % day)) * calendar[s] for s in score2)) # Maximize Rest to be towards beg of cycle


## Solver

### Solver parameters

In [68]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 1000 #When I increase this value for larger cycles I notice less clustering
solver.parameters.search_branching = cp_model.FIXED_SEARCH # follow decision strategy exactly
solver.parameters.enumerate_all_solutions = True
# solver.parameters.linearization_level = 0
# solver.parameters.search_branching = 5
# solver.parameters.search_branching = cp_model.PORTFOLIO_SEARCH
# solver.max_sat_reverse_assumption_order = 0

In [69]:
# Register solutions callback
class RunPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, calendar, day, cycle, session, solution_limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._calendar = calendar
        self._day = day
        self._cycle = cycle
        self._session = session
        self._solution_count = 0
        self._solution_limit = solution_limit
    
    def on_solution_callback(self):
        self._solution_count += 1
        for c in range(self._cycle):
            print(f'cycle:{c}')
            for d in range(self._day): 
                for s in range(len(self._session)):
                    if self.Value(self._calendar[(c, d, s)]):
                        print(f'\t day:{(d):<5d} {self._session[s]}')
        if self._solution_count >= self._solution_limit:
            print(f'solution limit reached {self._solution_count}')
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

In [70]:
solution_limit = 1
callback = RunPartialSolutionPrinter(calendar, day, cycle, session, solution_limit)

In [71]:

status = solver.Solve(model, callback)

cycle:0
	 day:0     rest
	 day:1     e 	 strides
	 day:2     e 	 strides
	 day:3     LR
	 day:4     e 	 strides
	 day:5     THRESH
	 day:6     e 	 hill
	 day:7     THRESH
	 day:8     e 	 hill
	 day:9     THRESH
cycle:1
	 day:0     rest
	 day:1     e 	 strides
	 day:2     e 	 strides
	 day:3     LR
	 day:4     e 	 strides
	 day:5     THRESH
	 day:6     e 	 hill
	 day:7     THRESH
	 day:8     e 	 hill
	 day:9     THRESH
cycle:2
	 day:0     rest
	 day:1     e 	 strides
	 day:2     LR
	 day:3     e 	 strides
	 day:4     THRESH
	 day:5     e 	 strides
	 day:6     THRESH
	 day:7     e 	 hill
	 day:8     THRESH
	 day:9     e 	 hill
solution limit reached 1


In [72]:
# solver.StatusName()
print(solver.ResponseStats())
# print(model.ModelStats())

CpSolverResponse summary:
status: OPTIMAL
objective: 30
best_bound: 30
booleans: 180
conflicts: 9
branches: 428
propagations: 1872
integer_propagations: 2282
restarts: 355
lp_iterations: 115
walltime: 0.0424932
usertime: 0.0424934
deterministic_time: 0.00133214
gap_integral: 0.00373362

